In [1]:
import numpy as np
import pandas as pd
import sklearn
import nltk
import re
import json

import keras
from keras.preprocessing import image

from tqdm import tqdm

Using TensorFlow backend.
/opt/anaconda3/envs/nmep/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/nmep/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/nmep/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/nmep/lib/python3.6/site-packag

In [2]:
movies = pd.read_csv('archive/movies_metadata.csv')
print("Shape: ", movies.shape)
print("Features: ", movies.columns)

Shape:  (45466, 24)
Features:  Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


/Users/harshikajalan/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [51]:
print("Shape: ", movies.shape)
print("Features: ", movies.columns)


Shape:  (45466, 24)
Features:  Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


In [4]:
movies['overview'][1]

"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures."

In [3]:
movies.head()['imdb_id']

0    tt0114709
1    tt0113497
2    tt0113228
3    tt0114885
4    tt0113041
Name: imdb_id, dtype: object

In [4]:
movies.head()['genres']

0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2    [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                       [{'id': 35, 'name': 'Comedy'}]
Name: genres, dtype: object

In [5]:
movies.head()['poster_path']

0    /rhIRbceoE9lR4veEXuwCC2wARtG.jpg
1    /vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg
2    /6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg
3    /16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg
4    /e64sOI48hQXyru7naBFyssKFxVd.jpg
Name: poster_path, dtype: object

In [6]:
movies.head()['genres'][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [7]:
movies.head()['genres'][1]

"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]"

In [8]:
# ARCHIVED FUNCTION
"""
def f(downloaded_image_dir, title, poster_path):
    if not os.path.exists(downloaded_image_dir):
        os.makedirs(downloaded_image_dir)
    
    # corrupted url
    imgUrl = "https://www.themoviedb.org/t/p/w1280" + str(poster_path)
    filename = re.sub(r'\W+', ' ', title).lower().strip().replace(" ", "-") + '.jpg'

    try:
        r = requests.get(imgUrl, stream=True)
        if r.status_code == 200:
            with open(downloaded_image_dir + '/' + filename, 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)
        else:
            print("Not Found: ", title, imgUrl)
        
    except:
        print("Error Dowloading: ", title, imgUrl)    
"""

'\ndef f(downloaded_image_dir, title, poster_path):\n    if not os.path.exists(downloaded_image_dir):\n        os.makedirs(downloaded_image_dir)\n    \n    # corrupted url\n    imgUrl = "https://www.themoviedb.org/t/p/w1280" + str(poster_path)\n    filename = re.sub(r\'\\W+\', \' \', title).lower().strip().replace(" ", "-") + \'.jpg\'\n\n    try:\n        r = requests.get(imgUrl, stream=True)\n        if r.status_code == 200:\n            with open(downloaded_image_dir + \'/\' + filename, \'wb\') as f:\n                r.raw.decode_content = True\n                shutil.copyfileobj(r.raw, f)\n        else:\n            print("Not Found: ", title, imgUrl)\n        \n    except:\n        print("Error Dowloading: ", title, imgUrl)    \n'

In [3]:
# @Source: https://johannesbader.ch/blog/tutorial-download-posters-with-the-movie-database-api-in-python/"

import os
import re
import requests

CONFIG_PATTERN = 'http://api.themoviedb.org/3/configuration?api_key={key}'
IMG_PATTERN = 'http://api.themoviedb.org/3/movie/{imdbid}/images?api_key={key}' 
KEY = '6f2ceb0e0b459afe90f6e854fcd410eb'
            
def _get_json(url):
    r = requests.get(url)
    return r.json()
    
def _download_images(titles, urls, path):
    """download all images in list 'urls' to 'path' """
    filePathList = []
    print(len(urls), len(titles))
    zipped = list(zip(titles, urls))
    for title, url in zipped:
        r = requests.get(url)
        filetype = r.headers['content-type'].split('/')[-1]
        filename = re.sub(r'\W+', ' ', title).lower().strip().replace(" ", "-") + '.jpg'
        filepath = os.path.join(path, filename)
        filePathList.append(filepath)
        
        if r.status_code == 200:
            with open(filepath,'wb') as w:
                w.write(r.content)
                
    return path, filePathList

def get_poster_url(imdbid):
    """ 
    given IMDB id, returns the corresponding url for
    poster jpg
    
    Args: 
        imdbid (str): IMDB id
    Return: 
        url (str): image url of poster image from 'themoviedb.org'
                of the maximum available size. 
    """
    config = _get_json(CONFIG_PATTERN.format(key=KEY))
    base_url = config['images']['base_url']
    sizes = config['images']['poster_sizes']   # sorted in ascending order
    
    def size_str_to_int(x):
        return float("inf") if x == 'original' else int(x[1:])
    
    # retrieve image of max size
    max_size = max(sizes, key=size_str_to_int)
    
    posters = _get_json(IMG_PATTERN.format(key=KEY,imdbid=imdbid))['posters']
    
    if len(posters) == 0:
        return None
    
    rel_path = posters[0]['file_path']
    url = "{0}{1}{2}".format(base_url, max_size, rel_path)
    
    return url

In [4]:
# RUN THIS FUNCTION TO DOWNLOAD JPG IMAGES
def tmdb_posters():
    """ download posters with imdb_id from dataset """
    
    if not os.path.exists('images_movies_genre'):
        os.makedirs('images_movies_genre')
    
    titles = []
    urls = []
    
    n = 0
    for index, row in movies.iterrows():
        title = str(row['title'])
        url = get_poster_url(row['imdb_id'])
        
        # skip iteration if poster cannot be located
        if not url:
            continue
            
        titles.append(title)
        urls.append(url)
        n += 1
        
        # SET LIMIT FOR NUMBER OF IMAGES TO DOWNLOAD
        if n > 20:
            break

    return _download_images(titles, urls, 'images_movies_genre')

path, filePathList = tmdb_posters() 

21 21


In [11]:
print(path)

images_movies_genre


In [12]:
print(filePathList)

['images_movies_genre/toy-story.jpg', 'images_movies_genre/jumanji.jpg', 'images_movies_genre/grumpier-old-men.jpg', 'images_movies_genre/waiting-to-exhale.jpg', 'images_movies_genre/father-of-the-bride-part-ii.jpg', 'images_movies_genre/heat.jpg', 'images_movies_genre/sabrina.jpg', 'images_movies_genre/tom-and-huck.jpg', 'images_movies_genre/sudden-death.jpg', 'images_movies_genre/goldeneye.jpg', 'images_movies_genre/the-american-president.jpg', 'images_movies_genre/dracula-dead-and-loving-it.jpg', 'images_movies_genre/balto.jpg', 'images_movies_genre/nixon.jpg', 'images_movies_genre/cutthroat-island.jpg', 'images_movies_genre/casino.jpg', 'images_movies_genre/sense-and-sensibility.jpg', 'images_movies_genre/four-rooms.jpg', 'images_movies_genre/ace-ventura-when-nature-calls.jpg', 'images_movies_genre/money-train.jpg', 'images_movies_genre/get-shorty.jpg']


In [5]:
# TODO: jpg --> numpy array conversion

X = []

for path in tqdm(filePathList):
    img = image.load_img(path, target_size=(350, 350, 3))
    img = image.img_to_array(img)
    img = img/255.0
    X.append(img)
    
X = np.array(X)

100%|██████████| 21/21 [00:01<00:00, 20.28it/s]


In [13]:
image.array_to_img(X[0]).show()

In [8]:
#Padding
#reference: https://ai-pool.com/d/padding-images-with-numpy
def pad(img, h, w):
    top_pad = np.floor((h - img.shape[0]) / 2).astype(np.uint16)
    bottom_pad = np.ceil((h - img.shape[0]) / 2).astype(np.uint16)
    right_pad = np.ceil((w - img.shape[1]) / 2).astype(np.uint16)
    left_pad = np.floor((w - img.shape[1]) / 2).astype(np.uint16)
    return np.copy(np.pad(img, ((top_pad, bottom_pad), (left_pad, right_pad), (0, 0)), mode='constant', constant_values=0))

In [14]:
#checking if padding works
img = image.load_img('images_movies_genre/toy-story.jpg', target_size=(350, 350, 3))
new_img_array=pad(X[0], 512, 512)
new_img=image.array_to_img(new_img_array)
new_img.show()

In [3]:
import ast
#reference-https://www.geeksforgeeks.org/adding-new-column-to-existing-dataframe-in-pandas/
def genre_list(str):
    arr=ast.literal_eval(str)
    genreList=[dict['name'] for dict in arr]
    return genreList

In [4]:
genre_list_collection=[]
for i in movies['genres']:
    genre_list_collection+=[genre_list(i)]
genre_list_collection[0]

['Animation', 'Comedy', 'Family']

In [5]:
#adding a column called genre_list which contains all the genres of the movies in a list
movies.insert(5,"genre_list", genre_list_collection, True)
print("Features: ", movies.columns)
movies['genre_list']

Features:  Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'genre_list', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45461                 [Drama, Family]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genre_list, Length: 45466, dtype: object

In [21]:
"""Removing genres that are very sparse in dataset"""

""" archived code because unwanted genres already decided below
dictSet = {}
totGenres = 0
for movie in movies['genre_list']: 
    for genre in movie: 
        if dictSet.get(genre, 0): 
            dictSet[genre] += 1
        else: 
            dictSet[genre] = 1
        totGenres += 1

dictSetIter = dictSet.copy()
for genre in dictSetIter: 
    if not (dictSet[genre]/(totGenres) < 0.01):
        print(genre)
        dictSet.pop(genre)

print()

for movie in movies['genre_list']: 
    movieIter = movie.copy()
    for genre in movieIter: 
        if dictSet.get(genre, 0):
            movie.remove(genre)
            print(genre)
"""

" archived code because unwanted genres already decided below\ndictSet = {}\ntotGenres = 0\nfor movie in movies['genre_list']: \n    for genre in movie: \n        if dictSet.get(genre, 0): \n            dictSet[genre] += 1\n        else: \n            dictSet[genre] = 1\n        totGenres += 1\n\ndictSetIter = dictSet.copy()\nfor genre in dictSetIter: \n    if not (dictSet[genre]/(totGenres) < 0.01):\n        print(genre)\n        dictSet.pop(genre)\n\nprint()\n\nfor movie in movies['genre_list']: \n    movieIter = movie.copy()\n    for genre in movieIter: \n        if dictSet.get(genre, 0):\n            movie.remove(genre)\n            print(genre)\n"

In [22]:
#TOD0: Remove unwanted genres
invalid_genres = ['Aniplex', 'BROSTA TV', 'Carousel Productions', 'GoHands', 'Mardock Scramble Production Committee', 'Odyssey Media', 'Pulser Productions', 'Rogue State', 'Sentai Filmworks', 'Telescene Film Group Productions', 'The Cartel', 'Vision View Entertainment', 'TV Movie', 'Foreign']
invalid_genres_dict = {}
for genre in invalid_genres: 
    invalid_genres_dict[genre] = 0
    
for movie in movies['genre_list']: 
    for genre in movie: 
        if invalid_genres_dict.get(genre, 0):
            movie.remove(genre)
            print(genre)

In [49]:
all_genres = sum(genre_list_collection,[])
len(set(all_genres))

20

In [6]:
#One hot encoding the genres
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(movies['genre_list'].values)
genreLabelsList=mlb.transform(movies['genre_list'])
print(genreLabelsList.shape)
#TODO Add the one hot encoded genres to the movies dataframe
genre_labels=pd.DataFrame(genreLabelsList, columns=mlb.classes_)
print(genre_labels)

(45466, 32)
       Action  Adventure  Animation  Aniplex  BROSTA TV  Carousel Productions  \
0           0          0          1        0          0                     0   
1           0          1          0        0          0                     0   
2           0          0          0        0          0                     0   
3           0          0          0        0          0                     0   
4           0          0          0        0          0                     0   
...       ...        ...        ...      ...        ...                   ...   
45461       0          0          0        0          0                     0   
45462       0          0          0        0          0                     0   
45463       1          0          0        0          0                     0   
45464       0          0          0        0          0                     0   
45465       0          0          0        0          0                     0   

       Comedy  

In [7]:
genre_labels.to_csv('GenreLabels.csv', index=False)

In [57]:
mlb.classes_

array(['Action', 'Adventure', 'Animation', 'Aniplex', 'BROSTA TV',
       'Carousel Productions', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Foreign', 'GoHands', 'History', 'Horror',
       'Mardock Scramble Production Committee', 'Music', 'Mystery',
       'Odyssey Media', 'Pulser Productions', 'Rogue State', 'Romance',
       'Science Fiction', 'Sentai Filmworks', 'TV Movie',
       'Telescene Film Group Productions', 'The Cartel', 'Thriller',
       'Vision View Entertainment', 'War', 'Western'], dtype=object)

In [58]:
movies['genre_list']

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45461                 [Drama, Family]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genre_list, Length: 45466, dtype: object

In [12]:
overviews=pd.DataFrame(movies["overview"])
movies_with_overviews=pd.concat([overviews, genre_labels], axis=1)
movies_with_overviews.head()
#movies_with_overveiws=genre_labels
#movies_with_overviews["overview"]=movies["overview"]

,overview,Action,Adventure,Animation,Aniplex,BROSTA TV,Carousel Productions,Comedy,Crime,Documentary,...,Romance,Science Fiction,Sentai Filmworks,TV Movie,Telescene Film Group Productions,The Cartel,Thriller,Vision View Entertainment,War,Western
0,"Led by Woody, Andy's toys live happily in his ...",0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,When siblings Judy and Peter discover an encha...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,A family wedding reignites the ancient feud be...,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
3,"Cheated on, mistreated and stepped on, the wom...",0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4,Just when George Banks has recovered from his ...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
print("Shape: ", movies_with_overviews.shape)
print("Features: ", movies_with_overviews.columns)
movies_with_overviews.to_csv('MoviesWithOverviews.csv', index=False)

Shape:  (45466, 33)
Features:  Index(['overview', 'Action', 'Adventure', 'Animation', 'Aniplex', 'BROSTA TV',
       'Carousel Productions', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Foreign', 'GoHands', 'History', 'Horror',
       'Mardock Scramble Production Committee', 'Music', 'Mystery',
       'Odyssey Media', 'Pulser Productions', 'Rogue State', 'Romance',
       'Science Fiction', 'Sentai Filmworks', 'TV Movie',
       'Telescene Film Group Productions', 'The Cartel', 'Thriller',
       'Vision View Entertainment', 'War', 'Western'],
      dtype='object')


In [61]:
movies.shape

(45466, 25)

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/harshikajalan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [24]:
# Text cleaning
#reference: https://www.analyticsvidhya.com/blog/2019/04/predicting-movie-genres-nlp-multi-label-classification/
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def clean_text(text): 
    #removing everything except letters and converting to lower case
    text = re.sub("[^a-zA-Z]"," ",str(text))
    text = ' '.join(text.split())
    text = text.lower() 
    return text

# Removing stopwords such as the, and, to, or etc
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

In [28]:
plot_summaries=[]
for text in movies['overview']:
    cleanText=clean_text(text)
    textWithoutStopwords=remove_stopwords(cleanText)
    plot_summaries+=[textWithoutStopwords]
movies.insert(11,"plot_summary", plot_summaries, True)
print("Features: ", movies.columns)

Features:  Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'plot_summary', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


In [29]:
movies["plot_summary"]

0        led woody andy toys live happily room andy bir...
1        siblings judy peter discover enchanted board g...
2        family wedding reignites ancient feud next doo...
3        cheated mistreated stepped women holding breat...
4        george banks recovered daughter wedding receiv...
                               ...                        
45461                             rising falling man woman
45462    artist struggles finish work storyline cult pl...
45463    one hits goes wrong professional assassin ends...
45464    small town live two brothers one minister one ...
45465    years decriminalisation homosexuality uk direc...
Name: plot_summary, Length: 45466, dtype: object